## How Python’s Memory Management Works

Python’s memory management is an important part of its performance and efficiency.       
 Understanding how Python handles memory can help developers write better code and       
 optimize their applications. This beginner-friendly article explores Python’s memory       
 management, covering reference counting, garbage collection, and memory allocation strategies.

# Reference Counting
https://medium.com/@AlexanderObregon/how-pythons-memory-management-works-f832405ea3a3

What is Reference Counting?    
--->  Reference counting is a memory management technique used by Python to keep track of the number of references to each object.  

--->  When an object’s reference count drops to zero, it means that the object is no longer in use and can be safely deallocated .  

---> This method is simple yet effective for many memory management scenarios and forms the backbone of Python’s memory management system.    

In [1]:
### Every Python object includes a reference count as part of its metadata. When you create a new object, its reference count is initialized to one.
# As you assign this object to different variables or data structures, the reference count increases.
#  Conversely, when you delete these references, the reference count decreases.


import sys

a = [1, 2, 3]
print(sys.getrefcount(a))  # Outputs 2 because the reference count includes the argument to getrefcount

b = a
print(sys.getrefcount(a))  # Outputs 3

c = a
print(sys.getrefcount(a))  # Outputs 4

del b
print(sys.getrefcount(a))  # Outputs 3

del c
print(sys.getrefcount(a))  # Outputs 2

del a
# The reference count drops to 1, but because sys.getrefcount was called, the object is still in scope within that call

2
3
4
3
2


### Handling Circular References
One of the main limitations of reference counting is    

its inability to handle circular references, where two or more objects reference each other, forming a cycle.   
In such cases, the reference count never drops to zero, leading to memory leaks.

Advantages of Reference Counting             
Immediate Reclamation: Memory is freed immediately when the reference count drops to zero, which can lead to lower memory usage and less overhead compared to other garbage collection strategies that may only reclaim memory at certain intervals.      
Simplicity: The implementation of reference counting is straightforward, making it easier to understand and debug.            
Deterministic Destruction: Objects are destroyed as soon as they are no longer needed, which can be particularly useful for managing resources like file handles or network connections.

..   
...   
...  
Disadvantages of Reference Counting   
Cyclic References: As mentioned, reference counting cannot handle cyclic references, which can lead to memory leaks.   
Performance Overhead: Incrementing and decrementing reference counts adds overhead to every assignment and deletion operation, which can impact performance,    especially in programs with many short-lived objects.   
Memory Overhead: Each object must store its reference count, adding to the memory footprint of objects.  

## Garbage Collection

--> Garbage collection (GC) is a mechanism that complements reference counting to  manage memory more effectively.     
--> It is used to detect and reclaim memory occupied by objects that are no longer accessible, even if they are part of a reference cycle.     
--> This is crucial for preventing memory leaks in applications where circular references might occur.  


---> The GC mechanism is part of Python’s gc module, which provides tools to inspect and manipulate the garbage collection process.



### How GC works--

==> 1. Generation based collection:

---> Python’s garbage collector divides objects into three generations based on their lifespan:

A. Newly created objects are placed in the first generation (young generation).  
B. Objects that survive garbage collection cycles are promoted to the second generation (middle generation)   

C. eventually to the third generation (old generation).


==> 2. Mark-and-Sweep Algorithm:
 use this algorithm to collect unreachable objects.   

 The GC traverses the objects graph starting from kown root objects(such as global variable and stack frames) then mark all as reachable objects.  

 collect al memory of unmarked objects, reclaiming their memory.


 ==> 3.   Thresholds and Triggers:

 GC trigger based on some thresholds, whihc are defined by the number of allocations and deallocations. when these threshold are axceeded, the garbage collector runs to clean up unused objects. 

 ---> these thresholds can be adjusted using GC module.




In [ ]:
import gc



gc.disable()  # Disable garbage collection
# Perform performance-critical operations
gc.enable()  # Re-enable garbage collection

In [ ]:
# Example of Tuning Garbage Collection
# Here is an example of how to use the gc module to control and inspect garbage collection in a Python application:

import gc

# Disable automatic garbage collection
gc.disable()

# Perform operations that generate a lot of temporary objects
temp_list = [i for i in range(10000)]
temp_list = None  # Remove the reference to the list

# Manually trigger garbage collection
gc.collect()

# Re-enable automatic garbage collection
gc.enable()

# Inspect the state of the garbage collector
print("Garbage collection thresholds:", gc.get_threshold())
print("Number of objects in each generation:", gc.get_count())


## memory allocation strategies

there is several memory allocation requests. these are designed to handle different types of memory requests, optimize performance and minimizing fragmentation.   

1. Row memory allocators:

--> thid directly interect with OS to allocate and deallocate memory bloacks. 

--> it is repsonsible for managing large memory requests and freeing memory back to system.

2. Object - Specific Allocators:

used for specific types of objects such as : Integer, strings, other build in types.

--> tailor memory to the object by the need of specific objects.   


3. Pymalloc:

---> special allocators designed for small objects (less than 512 bytes) 

--> enhance performance by reducing overhead associated with frequent allocation and deallocation.


Arenas, Pools, and Blocks   
Python’s memory management system organizes memory into arenas, pools, and blocks to manage different sizes of memory requests:     

Arenas: Large contiguous memory regions (256 KB each) allocated by the raw memory allocator. Each arena is subdivided into smaller pools.     

Pools: Fixed-size memory blocks (4 KB each) within an arena, used to manage small objects. Each pool is dedicated to objects of a specific size class.   

Blocks: The smallest units of memory within a pool, where individual objects are allocated. Each block is of a fixed size, corresponding to the size class of its pool.